In [ ]:
#@title Installing foundry:
!curl -L https://foundry.paradigm.xyz | bash
!source ~/.bashrc
import os
os.environ['PATH'] += ":/root/.foundry/bin"
!foundryup

# cloning repo and installing foundry dependencies:
!git clone -b feat/scripts-and-docs https://github.com/baking-bad/etherlink-bridge.git
%cd etherlink-bridge/
!git submodule update --init --recursive

# installing python dependencies:
!pip install poetry
!poetry install

### Environment Initialization:

Run the following cell to set up the environment and provide private keys for interacting *with* the bridge. Default values are provided for all variables; however, you may wish to deploy your own token and establish a bridge using your own keys.

In [16]:
!poetry run init_wallets

Enter "L1_PRIVATE_KEY" (default: edsk2nu78mRwg4V5Ka7XCJFVbVPPwhry8YPeEHRwzGQHEpGAffDvrH): 
Enter "L1_PUBLIC_KEY_HASH" (default: tz1YG6P2GTQKFpd9jeuESam2vg6aA9HHRkKo): 
Enter "L1_RPC_URL" (default: https://rpc.tzkt.io/nairobinet/): 
Enter "L1_ROLLUP_ADDRESS" (default: sr1Cz5HkqyC8ozTYbDij8YmseAAG3SgZtpRb): sr19t5DDEhvnPaNZgNYVAkmBRd5nAPyMU7nP
Enter "L2_PRIVATE_KEY" (default: 8636c473b431be57109d4153735315a5cdf36b3841eb2cfa80b75b3dcd2d941a): 
Enter "L2_PUBLIC_KEY" (default: 0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448): 
Enter "L2_MASTER_KEY" (default: 9722f6cc9ff938e63f8ccb74c3daa6b45837e5c5e3835ac08c44c50ab5f39dc0): 
Enter "L2_RPC_URL" (default: https://etherlink.dipdup.net): 
Enter "L2_KERNEL_ADDRESS" (default: 0x0000000000000000000000000000000000000000): 
Enter "L2_WITHDRAW_PRECOMPILE_ADDRESS" (default: 0x0000000000000000000000000000000000000040): 


### Bridge token pair listing:

The **Ticket Transport Layer**, a key component of the bridge, enables ticket transfers between Tezos and Etherlink. To list a new token pair and establish a bridge, users need to:
1. If the token is an **FA2** or **FA1.2** standard token that doesn’t natively support tickets, the user must deploy a **Ticketer** contract on the Tezos side. This contract links to the specific Tezos token to convert it into a ticket.
2. The user then deploys an **ERC20Proxy** on the Etherlink side – a smart contract implementing ERC20. This contract should be configured to expect tickets from the deployed **Ticketer** including ticketer address and ticket content. This setup allows the rollup to mint ERC20 tokens corresponding to the incoming tickets from Tezos.
3. Additionally, deploy a **TicketHelper** on the Tezos side, targeting the specific token and **Ticketer** pair. This step is necessary as the **transfer_ticket** operation is not currently supported by wallets. The **TicketHelper** facilitates the wrapping of tokens into tickets and their transfer to the rollup in a single transaction.

#### Deploying a Token

For demonstration purposes, users can deploy a test token that will later be bridged. The bridge has been tested with two types of tokens, which are available in the repository:
- The **FA1.2** **Ctez** token.
- The **FA2** **fxhash** token.

To deploy this token and allocate the total supply to the token's originator, execute the following script. The example below illustrates the deployment for an **FA1.2** token type:

In [5]:
!poetry run deploy_token --token-type FA1.2

deploying contract from filename /content/etherlink-bridge/tezos/tests/helpers/../tokens/fa12-Ctez.tz
found contract address: KT1GM2AnBAJWdzrChp3hTYFTSb6Dmh61peBP


#### Deploying a Ticketer

To deploy a ticketer for a specific token address and token id, use the following script. The example below uses a **FA1.2** token previously deployed on Nairobinet:

In [6]:
!poetry run deploy_ticketer --token-address KT1GM2AnBAJWdzrChp3hTYFTSb6Dmh61peBP --token-type FA1.2 --token-id 0

deploying contract from filename /content/etherlink-bridge/tezos/tests/helpers/../../build/ticketer.tz
found contract address: KT1PmYUomF3HDxsGWYQUCbLi2X8WvT7ZHv8o
address_bytes: 01a6938a03cf1d7652a7b871bd9c7c36b4655fa80300
content_bytes: 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a000000160155275f943ba305902a52dbf648236db528e0f015000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32


During the Ticketer's deployment, user will receive its parameters, including **address_bytes** and **content_bytes**. These are required for the origination of the **ERC20Proxy**.

#### Deploying a Ticket Helper

To enable Tezos wallets to interact with specific tickets, deploy a Ticket Helper using the script below. The example provided uses a ticketer that was previously deployed in Nairobinet:

In [7]:
!poetry run deploy_ticket_helper --ticketer-address KT1PmYUomF3HDxsGWYQUCbLi2X8WvT7ZHv8o

deploying contract from filename /content/etherlink-bridge/tezos/tests/helpers/../../build/ticket-helper.tz
found contract address: KT1TZg9EwGHKbfWvsHGsqBjm3J5NhJBtHPKX


#### Deploying ERC20Proxy

Finally, to deploy a token contract on the Etherlink side, which will mint tokens upon deposit, execute the script below. This script requires **ticketer-address-bytes** and **ticketer-content-bytes**. The example uses bytes from a ticketer previously deployed in Nairobinet:

In [10]:
!poetry run deploy_erc20 --ticketer-address-bytes 01a6938a03cf1d7652a7b871bd9c7c36b4655fa80300 --ticketer-content-bytes 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a000000160155275f943ba305902a52dbf648236db528e0f015000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32 --token-name "FA1.2 Test Token" --token-symbol "FA1.2" --decimals 0

Successfully deployed ERC20 contract:
Compiling 29 files with 0.8.21
Solc 0.8.21 finished in 9.72s
Compiler run successful!
Deployer: 0xBefD2C6fFC36249ebEbd21d6DF6376ecF3BAc448
Deployed to: 0x87dcBf128677ba36E79D47dAf4eb4e51610e0150
Transaction hash: 0xbcc95d28cc4d0dc8831a54a3c28515d2f7ddd4f6a349438aea0769c922632819



NOTE: To obtain the **ticketer-address-bytes** and **content-bytes** for an already deployed ticketer, user can use the script below:

In [12]:
!poetry run get_ticketer_params --ticketer KT1PmYUomF3HDxsGWYQUCbLi2X8WvT7ZHv8o

address_bytes: 01a6938a03cf1d7652a7b871bd9c7c36b4655fa80300
content_bytes: 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a000000160155275f943ba305902a52dbf648236db528e0f015000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32


### Deposit

To make a deposit, user need to transfer Tickets to the rollup address with attached Routing Info in the [specified format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#deposit): `| receiver | proxy |` 40 bytes payload, both receiver and proxy are standard Ethereum addresses in raw form (H160). A script is available to facilitate this process. It requires the Ticket Helper address as **ticket-helper-address**, the Etherlink ERC20Proxy contract as **proxy-address**, and the bridged amount as the **amount** variable. Here's an example:

In [17]:
!poetry run deposit --ticket-helper-address KT1TZg9EwGHKbfWvsHGsqBjm3J5NhJBtHPKX --proxy-address 0x87dcBf128677ba36E79D47dAf4eb4e51610e0150 --amount 42

Succeed, transaction hash: ooKv5MfH6s4RV4QSMHA5Xcwh4pGGncdLjidRXMakfVB1Jo4SPDB


NOTE: This script performs two operations in bulk. The first operation approves the token for the **Ticket Helper**, and the second operation makes a call to the **Ticket Helper** **deposit** entrypoint.

### Withdrawal Process

The withdrawal process involves two key steps:
1. Initiating the withdrawal on the Etherlink side, which results in the creation of an outbox message in the rollup commitment.
2. Completing the withdrawal by executing the outbox message on the Tezos side once the commitment has settled (TODO: Clarify terminology if necessary).

#### Etherlink Withdrawal

To initiate a withdrawal, user need to call the withdrawal precompile on the Etherlink side. This requires providing the **ERC20Proxy** address along with Routing Info in a [specific format](https://gitlab.com/baking-bad/tzip/-/blob/wip/029-etherlink-token-bridge/drafts/current/draft-etherlink-token-bridge/etherlink-token-bridge.md#withdrawal), two forged contracts concatenated: `| receiver | proxy |` 44 bytes. Forged contact consists of binary suffix/prefix and body (blake2b hash digest). A script is available to facilitate the withdrawal process. It requires the ERC20 contract (which will burn tokens) as **proxy-address**, the Tezos router address (which will receive the ticket from the rollup) as **router-address**, and the bridged amount as **amount**. Additionally, the **ticketer-address-bytes** and **ticketer-content-bytes** are required to allow **ERC20Proxy** validate token before burning.

NOTE: For automatic unwrapping of Tezos tickets back into tokens, the **Ticketer** address can be provided as the **router-address**.

In [19]:
!poetry run withdraw --proxy-address 0x87dcBf128677ba36E79D47dAf4eb4e51610e0150 --amount 17 --router-address KT1PmYUomF3HDxsGWYQUCbLi2X8WvT7ZHv8o --ticketer-address-bytes 01a6938a03cf1d7652a7b871bd9c7c36b4655fa80300 --ticketer-content-bytes 0707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a000000160155275f943ba305902a52dbf648236db528e0f015000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32

Successfully called withdraw:

blockHash               0x9be216703a76e3f32d54cccf2f762eb5ec3c3bc2739068a0a64764f5d5cd90cf
blockNumber             1990
contractAddress         
cumulativeGasUsed       29857
effectiveGasPrice       1
gasUsed                 29857
logs                    [{"address":"0x0000000000000000000000000000000000000000","topics":["0x64e765b73c0829c01c6f3026a840120142d5a0b83626c548cff3a44ca5299c98","0xbc8b3231d0cea743d39da0f658f08ca00e33affabc3d71b74a50d2dd2bacdde2"],"data":"0x000000000000000000000000befd2c6ffc36249ebebd21d6df6376ecf3bac44800000000000000000000000087dcbf128677ba36e79d47daf4eb4e51610e015000008a7390072a389159c73687165cd7910e8a3916060000000000000000000000000000000000000000000000000000000000000000000000000000000000110000000000000000000000000000000000000000000000000000000000291e060000000000000000000000000000000000000000000000000000000000000000","blockHash":"0x9be216703a76e3f32d54cccf2f762eb5ec3c3bc2739068a0a64764f5d5cd90cf","blockNumber":"0x7c6","transact

#### Finalizing Tezos Withdrawal

To complete the withdrawal process, user need to call the outbox message on the Tezos side once it has been finalized (settled) on the L1 side. During the withdrawal process, the Etherlink side emits a **Withdrawal** event, which includes **outboxLevel** and **outboxMsgId**.

TODO: Provide a link to the Blockscout page featuring this event.

TODO: Explain how to parse this information.

In [24]:
!poetry run get_proof --level 2694662 --index 0 --rpc-url https://etherlink-rollup-nairobi.dipdup.net

proof: {'commitment': 'src13WCyTUhs2uQ4Dao3uPdyDTHamdunfUpohH9iVWBh4kM7BNufn4', 'proof': '03000228e1b5375e60ad38f3fccbda58d3036f0b5d9c17377bf04a57f2c21ee999474728e1b5375e60ad38f3fccbda58d3036f0b5d9c17377bf04a57f2c21ee99947470005820764757261626c65d020be46d6fee780f11d6ad3d751aae6166a8609f8d594b7d5b73ee124574db6db03746167c00800000004536f6d650003c0b053c0f736c4449c6e34b05f6354314b9c1cb89d5cc5a7034fda8301ea21d230820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc00400291e9b0133810f76616c69646974795f706572696f64c00400013b0082013181086f7574626f78657301085c01043ac048a801cb804938afa19fa802dc4338c0cdb62f3af4181f1af21552d126aa389401021fc0d2a3335d2f4f53e37f2eea53c8a983456aa12df5f0a10d20ce5115482f1abf2a01010ec0f391ac26a76694ef64d154f6c724eaa943137db30c09ea9d7ffcac8e17e70e21008b0051002b0017c0e5273ca8a1c2b6b93f23aaf90631e387d1c44a0a1d9443e01fe27c89505474fd000d000700050003820732363932363931820468656164c00100066c

For executing the outbox message, run:

In [25]:
!poetry run execute_outbox_message --commitment src13WCyTUhs2uQ4Dao3uPdyDTHamdunfUpohH9iVWBh4kM7BNufn4 --proof 03000228e1b5375e60ad38f3fccbda58d3036f0b5d9c17377bf04a57f2c21ee999474728e1b5375e60ad38f3fccbda58d3036f0b5d9c17377bf04a57f2c21ee99947470005820764757261626c65d020be46d6fee780f11d6ad3d751aae6166a8609f8d594b7d5b73ee124574db6db03746167c00800000004536f6d650003c0b053c0f736c4449c6e34b05f6354314b9c1cb89d5cc5a7034fda8301ea21d230820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc00400291e9b0133810f76616c69646974795f706572696f64c00400013b0082013181086f7574626f78657301085c01043ac048a801cb804938afa19fa802dc4338c0cdb62f3af4181f1af21552d126aa389401021fc0d2a3335d2f4f53e37f2eea53c8a983456aa12df5f0a10d20ce5115482f1abf2a01010ec0f391ac26a76694ef64d154f6c724eaa943137db30c09ea9d7ffcac8e17e70e21008b0051002b0017c0e5273ca8a1c2b6b93f23aaf90631e387d1c44a0a1d9443e01fe27c89505474fd000d000700050003820732363932363931820468656164c00100066c656e677468c0010007323639343636320003810468656164c001008208636f6e74656e7473810130c0d3000000cf00000000ca07070a0000001600008a7390072a389159c73687165cd7910e8a39160607070a0000001601a6938a03cf1d7652a7b871bd9c7c36b4655fa8030007070707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a000000160155275f943ba305902a52dbf648236db528e0f015000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32001101a6938a03cf1d7652a7b871bd9c7c36b4655fa80300000000087769746864726177066c656e677468c00101c022af921df952e617e66234e21690f2006bacd3a1760199b0f5e3e07e9bff2d8ac0ebc21a40cf346f2ac3cb42e4ef52f86e7cc692aa1a3d378e86a24d352e1c47a9c006157fb9a642b6f089c639b40a175b594d9438f833118cf3ae7af81abd122aafc0aa94b566598f71fd31bcf22b6d4cf669373aac2069915eb6ba8db11069e29bffc00215ae56f089ef5991434b184238103a79caa6c1ced69a0ce85c5e1784fe1998c0488a5bf6d6c6e55ae425f5f5707637963f2aee3e314a9b8a91a0fc67c1155e82c02f3c2a933825a9103532296e6146bea9807b08b328f1f59201210dbffbd43976c03e39e3adf52cc5f0cb761e03985323d165382002999b96462fa5197ac010eb030134810d6d6573736167655f6c696d6974c002a401047761736dd03bd2249b38d061259e0347d92ac3104ab1448b2aa31ce47401700f14ced82c9528e1b5375e60ad38f3fccbda58d3036f0b5d9c17377bf04a57f2c21ee999474700291e060000000000ca07070a0000001600008a7390072a389159c73687165cd7910e8a39160607070a0000001601a6938a03cf1d7652a7b871bd9c7c36b4655fa8030007070707000005090a0000005f05020000005907040100000010636f6e74726163745f616464726573730a0000001c050a000000160155275f943ba305902a52dbf648236db528e0f015000704010000000a746f6b656e5f747970650a0000000b0501000000054641312e32001101a6938a03cf1d7652a7b871bd9c7c36b4655fa80300000000087769746864726177

Command not found: execute_outbox_message


### Other scripts:

- Fund account:

In [4]:
!poetry run fund_etherlink_account

Successfully funded account:

blockHash               0x56eba9a1f53fefc4de287edec978669c5d27413d0f8f430482758d8baa7da4b6
blockNumber             1766
contractAddress         
cumulativeGasUsed       21000
effectiveGasPrice       1
gasUsed                 21000
logs                    []
logsBloom               0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
root                    
status                  1
transactionHash         0xae53190694f0943e5f8b2cb28a0ed82156cc1b198d54a762fa7d25116e0f0359
transactionIndex        0
type 